In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-turbo",
#     tools=functions,
# )
assistant_id = "asst_5pa9C9Kw2rsf1BIVDSqN6SZt"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy.",
        }
    ]
)
print(thread)

Thread(id='thread_y6FbiZ63sFwoka6adn1OF5Ta', created_at=1726411104, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [24]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

print(run)

Run(id='run_Mmr0OmehnaTrRn09RP6g0BZX', assistant_id='asst_5pa9C9Kw2rsf1BIVDSqN6SZt', cancelled_at=None, completed_at=None, created_at=1726411108, expires_at=1726411708, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_y6FbiZ63sFwoka6adn1OF5Ta', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefini

In [21]:
get_run(run.id, thread.id).status

'completed'

In [17]:
print(get_run(run.id, thread.id).required_action.to_dict())

{'submit_tool_outputs': {'tool_calls': [{'id': 'call_ond6PW3VQp5EDF23bzRtUVg0', 'function': {'arguments': '{"ticker": "CRM"}', 'name': 'get_income_statement'}, 'type': 'function'}, {'id': 'call_yzyhJWu05ieHTNWLG7nsDczn', 'function': {'arguments': '{"ticker": "CRM"}', 'name': 'get_balance_sheet'}, 'type': 'function'}, {'id': 'call_yHknw87gtbBDI7xJUpgc6GXO', 'function': {'arguments': '{"ticker": "CRM"}', 'name': 'get_daily_stock_performance'}, 'type': 'function'}]}, 'type': 'submit_tool_outputs'}


In [18]:
# run의 status가 requires_action일 때, tool_output을 전달하기 위해 tool_output을 만드는 부분
tool_outputs = [
    {
        "tool_call_id": tool_call["id"],
        "output": globals()[tool_call["function"]["name"]](
            json.loads(tool_call["function"]["arguments"]),
        ),
    }
    for tool_call in get_run(run.id, thread.id).required_action.to_dict()[
        "submit_tool_outputs"
    ]["tool_calls"]
]
tool_outputs

[{'tool_call_id': 'call_ond6PW3VQp5EDF23bzRtUVg0',
  'output': '"{\\"1706659200000\\":{\\"Tax Effect Of Unusual Items\\":-202400000.0,\\"Tax Rate For Calcs\\":0.16,\\"Normalized EBITDA\\":11223000000.0,\\"Total Unusual Items\\":-1265000000.0,\\"Total Unusual Items Excluding Goodwill\\":-1265000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":4136000000.0,\\"Reconciled Depreciation\\":3959000000.0,\\"Reconciled Cost Of Revenue\\":8541000000.0,\\"EBITDA\\":9958000000.0,\\"EBIT\\":5999000000.0,\\"Normalized Income\\":5198600000.0,\\"Net Income From Continuing And Discontinued Operation\\":4136000000.0,\\"Total Expenses\\":28858000000.0,\\"Total Operating Income As Reported\\":5011000000.0,\\"Diluted Average Shares\\":984000000.0,\\"Basic Average Shares\\":974000000.0,\\"Diluted EPS\\":4.2,\\"Basic EPS\\":4.25,\\"Diluted NI Availto Com Stockholders\\":4136000000.0,\\"Net Income Common Stockholders\\":4136000000.0,\\"Net Income\\":4136000000.0,\\"Net Income Including N

In [19]:
# tool_outputs를 제출하기 위한 코드
client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id, thread_id=thread.id, tool_outputs=tool_outputs
)

Run(id='run_Mmr0OmehnaTrRn09RP6g0BZX', assistant_id='asst_5pa9C9Kw2rsf1BIVDSqN6SZt', cancelled_at=None, completed_at=None, created_at=1726411108, expires_at=1726411708, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726411247, status='queued', thread_id='thread_y6FbiZ63sFwoka6adn1OF5Ta', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Function

In [25]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy.
assistant: Considering Salesforce's financial and stock performance data, let's evaluate if their stock is a good buy:

### 1. Financial Performance Analysis
#### Income Statement Highlights:
- **Total Revenue**: Grew from $26.492 billion in 2020 to $34.857 billion in 2022, indicating a strong growth trajectory.
- **Operating Income**: Increased significantly to $5.999 billion in 2022 from just $548 million in 2020.
- **Net Income**: Also saw substantial growth, from $1.444 billion in 2020 to $4.136 billion in 2022.

#### Balance Sheet Highlights:
- **Total Assets**: Increased from $66.3 billion in 2020 to $99.8 billion in 2022.
- **Total Liabilities**: Grew from $24.8 billion in 2020 to $40.1 billion in 2022, but this is proportionate to asset and equity growth.
- **Stockholders' Equity**: Consistently rising, from $41.5 billion in 2020 to $59.6 billion in 2022, indicating strengthening financial stability.

### 2. Recent Sto